In [6]:
!pip install obonet
!pip install pandas

DEPRECATION: o365 2.0.16 has a non-standard dependency specifier tzlocal<3.*,>=1.5.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of o365 or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: o365 2.0.16 has a non-standard dependency specifier tzlocal<3.*,>=1.5.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of o365 or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Import the necessary libraries
import os
import obonet
import pandas as pd

folder_source = 'rawdl_20240310'
ontology_file = 'hp.obo'
hpo_file = 'genes_to_phenotype.txt'

### Subsetting the HPO that contain external reference to SNOMED-CT

In [8]:
# Read the ontology file
graph = obonet.read_obo(os.path.join(folder_source, ontology_file))

# Mapping from term ID to xref with prefix of SNOMEDCT_US
id_to_xref = {id_: data.get('xref') for id_, data in graph.nodes(data=True)}

# Iterate through all HPO code, name, and all the snomed ct ID from xref, 
# if there are multiple SNOMED CT ID, create multiple rows
# save to pandas dataframe file and tsv file

# Loop all the HPO code and name, and all the snomed ct ID from xref
hpo_snomedct_us = []
for hpo_id, xrefs in id_to_xref.items():
    # If there are no xrefs, append None to the list
    hpo_name = graph.nodes[hpo_id]['name']
    if xrefs is None:
        hpo_snomedct_us.append((hpo_id, hpo_name, None))
        print(f'{hpo_id} has no xrefs')
        continue

    # Check if there are xrefs related to SNOMED-CT,if there are at least one, create a row for SNOMED-CT, and skip the UMLS or other part
    # Else : return None
    check_snomedct = 0
    for xref in xrefs:
        if 'SNOMEDCT_US' in xref:
            snomed_ct_id = xref.split(':')[-1]
            hpo_snomedct_us.append((hpo_id, hpo_name, snomed_ct_id))
            check_snomedct += 1
        else:
            continue

    if check_snomedct == 0:
        hpo_snomedct_us.append((hpo_id, hpo_name, None))
        print(f'{hpo_id} has no SNOMEDCT_US xrefs')

# Save to pandas dataframe and tsv file
hpo_snomedct_us_df = pd.DataFrame(hpo_snomedct_us, columns=['HPO_ID', 'HPO_NAME', 'SNOMED_CT_ID'])
hpo_snomedct_us_df.to_csv(os.path.join('subset', 'hpo2snomed_subset.tsv'), sep='\t', index=False)

HP:0000001 has no SNOMEDCT_US xrefs
HP:0000002 has no SNOMEDCT_US xrefs
HP:0000005 has no SNOMEDCT_US xrefs
HP:0000008 has no SNOMEDCT_US xrefs
HP:0000009 has no SNOMEDCT_US xrefs
HP:0000014 has no SNOMEDCT_US xrefs
HP:0000021 has no SNOMEDCT_US xrefs
HP:0000022 has no SNOMEDCT_US xrefs
HP:0000025 has no SNOMEDCT_US xrefs
HP:0000030 has no SNOMEDCT_US xrefs
HP:0000032 has no SNOMEDCT_US xrefs
HP:0000033 has no SNOMEDCT_US xrefs
HP:0000036 has no SNOMEDCT_US xrefs
HP:0000042 has no SNOMEDCT_US xrefs
HP:0000045 has no SNOMEDCT_US xrefs
HP:0000049 has no SNOMEDCT_US xrefs
HP:0000050 has no SNOMEDCT_US xrefs
HP:0000052 has no SNOMEDCT_US xrefs
HP:0000053 has no SNOMEDCT_US xrefs
HP:0000055 has no SNOMEDCT_US xrefs
HP:0000056 has no SNOMEDCT_US xrefs
HP:0000058 has no SNOMEDCT_US xrefs
HP:0000060 has no SNOMEDCT_US xrefs
HP:0000061 has no SNOMEDCT_US xrefs
HP:0000063 has no SNOMEDCT_US xrefs
HP:0000064 has no SNOMEDCT_US xrefs
HP:0000066 has no SNOMEDCT_US xrefs
HP:0000067 has no SNOMEDCT_U

In [9]:
# Read the genes_to_phenotype file, split it to ORPHA and OMIM and save to files
genes_to_phenotype = pd.read_csv(os.path.join(folder_source, hpo_file), sep='\t')

# read genes_to_phenotype and filter the disease_id having prefix ORPHA:
orpha_to_hpo = genes_to_phenotype[genes_to_phenotype['disease_id'].str.contains('ORPHA:')]
orpha_to_hpo = orpha_to_hpo.drop_duplicates()
orpha_to_hpo.to_csv(os.path.join('subset', 'orpha2hpo_subset.tsv'), sep='\t', index=False)

# read genes_to_phenotype and filter the disease_id having prefix OMIM:
omim_to_hpo = genes_to_phenotype[genes_to_phenotype['disease_id'].str.contains('OMIM:')]
omim_to_hpo = omim_to_hpo.drop_duplicates()
omim_to_hpo.to_csv(os.path.join('subset', 'omim2hpo_subset.tsv'), sep='\t', index=False)